# Join by indetifier

In [2]:
import geopandas as gpd
import pandas as pd

# Load the census data from an Excel file
#census_df = pd.read_excel('/Users/yingxizhao/Library/CloudStorage/GoogleDrive-yiz158@ucsd.edu/Shared drives/Econ Research Lab Fall 2023/Data/census/census data/census blocks/utah_block_2010.xlsx')
census_df = gpd.read_file('/Users/yingxizhao/Library/CloudStorage/GoogleDrive-yiz158@ucsd.edu/Shared drives/Econ Research Lab Fall 2023/Data/census/shapefiles/2010 blocks/Utah 2010/tl_2010_49_tabblock10.shp')
# Convert the DataFrame to a GeoDataFrame
# census_gdf = gpd.GeoDataFrame(census_df)

# Load the shapefile for wards
wards = gpd.read_file('/Users/yingxizhao/Library/CloudStorage/GoogleDrive-yiz158@ucsd.edu/Shared drives/Econ Research Lab Fall 2023/Data/ward boundaries/slc_area_2013_cleaned.shp')

# Assuming that both `census_gdf` and `wards` have the 'GEOID' column
# Use a spatial join to find intersections based on 'GEOID'
overlap_gdf = gpd.sjoin(wards, census_df, how="inner", op='intersects')

# Create a GeoDataFrame to store the results with the correct geometry
results = gpd.GeoDataFrame(columns=['overlap_fractions'], geometry=wards.geometry)

# Iterate through each ward polygon
for ward_index, ward_row in wards.iterrows():
    # Note: Assuming 'GEOID' is the column to match on
    ward_geoid = ward_row['GEOID']
    
    # Filter overlaps based on the 'GEOID'
    # Note: You need to join on some attribute, 'GEOID' is assumed to be present in both dataframes
    matching_rows = overlap_gdf[overlap_gdf['GEOID'] == ward_geoid]
    
    # Calculate overlap fractions for each match
    overlap_fractions = [row['geometry'].intersection(ward_row['geometry']).area / ward_row['geometry'].area for index, row in matching_rows.iterrows()]
    
    # Add the sum of overlap fractions for this ward to the results GeoDataFrame
    results.at[ward_index, 'overlap_fractions'] = sum(overlap_fractions)

# Now 'results' already has a geometry column because we initialized it with one,
# so we can save it directly
results.to_file('ward_census_overlap.shp')


DriverError: path_to_census_shapefile.shp: No such file or directory

In [5]:
import geopandas as gpd

# Load the census data as a GeoDataFrame
census_df = gpd.read_file('/Users/yingxizhao/Library/CloudStorage/GoogleDrive-yiz158@ucsd.edu/Shared drives/Econ Research Lab Fall 2023/Data/census/shapefiles/2010 blocks/Utah 2010/tl_2010_49_tabblock10.shp')

# Load the shapefile for wards as a GeoDataFrame
wards = gpd.read_file('/Users/yingxizhao/Library/CloudStorage/GoogleDrive-yiz158@ucsd.edu/Shared drives/Econ Research Lab Fall 2023/Data/ward boundaries/slc_area_2013_cleaned.shp')

# ... [previous code remains the same]

# Use a spatial join to find intersections between wards and census blocks
overlap_gdf = gpd.sjoin(census_df, wards, how="inner", op='intersects')

# Initialize an empty list to hold the overlap fractions
overlap_fractions = []

# Iterate through each ward polygon
for ward_index, ward_row in wards.iterrows():  # Changed `wards_gdf` to `wards`
    # Get the current ward's geometry
    ward_geom = ward_row.geometry
    
    # Initialize the overlap fraction for this ward
    ward_overlap_fraction = 0
    
    # Find the census blocks that intersect with this ward
    intersecting_blocks = overlap_gdf[overlap_gdf.index_right == ward_index]  # Changed `index_left` to `index_right`
    
    # Sum the area of intersection with all intersecting census blocks
    for block_index, block_row in intersecting_blocks.iterrows():
        intersection_area = ward_geom.intersection(block_row.geometry).area
        ward_overlap_fraction += intersection_area
    
    # Normalize the overlap fraction by the area of the ward
    ward_overlap_fraction /= ward_geom.area
    
    # Append the result to the list
    overlap_fractions.append(ward_overlap_fraction)

# Create a new GeoDataFrame to store the results
results_gdf = wards.copy()  # Changed `wards_gdf` to `wards`
results_gdf['overlap_fractions'] = overlap_fractions

# Save the results to a new shapefile
results_gdf.to_file('ward_census_overlap.shp')

/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/pw/0gqphwys0t16_25rb5s6kp800000gn/T/ipykernel_82604/3991555311.py:12: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4269
Right CRS: EPSG:3857

  overlap_gdf = gpd.sjoin(census_df, wards, how="inner", op='intersects')
/var/folders/pw/0gqphwys0t16_25rb5s6kp800000gn/T/ipykernel_82604/3991555311.py:44: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  results_gdf.to_file('ward_census_overlap.shp')


In [3]:
# use GEOID to match census and shapefiles
# no shared identifier
print(census_df.columns)
wards

Index(['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'BLOCKCE10', 'GEOID10',
       'NAME10', 'MTFCC10', 'UR10', 'UACE10', 'UATYP10', 'FUNCSTAT10',
       'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'geometry'],
      dtype='object')


,VALUE,x_min,stake_id,ward_id,id,layer,path,_errors,geometry
0,0.0,-1.246956e+07,271594,193771,None,271594,/Users/kurtisgilliat/Dropbox/Data/Ward Boundar...,Self-intersection,"POLYGON ((-12467150.764 4948534.872, -12467188..."
1,NaN,NaN,482277,364703,None,482277,/Users/kurtisgilliat/Dropbox/Data/Ward Boundar...,Ring self-intersection,"POLYGON ((-12468857.556 4937069.541, -12468867..."
2,0.0,-1.245209e+07,503207,3700,None,503207,/Users/kurtisgilliat/Dropbox/Data/Ward Boundar...,Ring self-intersection,"POLYGON ((-12446533.013 4975582.519, -12446604..."
3,NaN,NaN,507253,104981,None,507253,/Users/kurtisgilliat/Dropbox/Data/Ward Boundar...,Self-intersection,"POLYGON ((-12250425.736 5033828.540, -12250246..."
4,0.0,-1.247227e+07,482226,453552,None,482226,/Users/kurtisgilliat/Dropbox/Data/Ward Boundar...,Ring self-intersection,"POLYGON ((-12470454.195 4931660.856, -12470624..."
...,...,...,...,...,...,...,...,...,...
1717,0.0,-1.245353e+07,618012,241156,None,618012,/Users/kurtisgilliat/Dropbox/Data/Ward Boundar...,None,"POLYGON ((-12453551.589 4954922.538, -12453550..."
1718,0.0,-1.245458e+07,618012,4588,None,618012,/Users/kurtisgilliat/Dropbox/Data/Ward Boundar...,None,"POLYGON ((-12454594.508 4955792.224, -12454594..."
1719,0.0,-1.245459e+07,618012,191159,None,618012,/Users/kurtisgilliat/Dropbox/Data/Ward Boundar...,None,"POLYGON ((-12454606.970 4955410.035, -12454606..."
1720,0.0,-1.245331e+07,618012,128961,None,618012,/Users/kurtisgilliat/Dropbox/Data/Ward Boundar...,None,"POLYGON ((-12453324.666 4955508.158, -12453335..."


In [4]:
census_df

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,49,001,100100,1115,490011001001115,Block 1115,G5040,R,None,None,S,811395,0,+38.4353985,-112.6160264,"POLYGON ((-112.61926 38.43065, -112.61934 38.4..."
1,49,001,100100,1173,490011001001173,Block 1173,G5040,R,None,None,S,1048041,0,+38.4197185,-112.6142793,"POLYGON ((-112.60139 38.42624, -112.60138 38.4..."
2,49,001,100100,1186,490011001001186,Block 1186,G5040,R,None,None,S,11643,0,+38.3930449,-112.5975317,"POLYGON ((-112.59804 38.39624, -112.59783 38.3..."
3,49,001,100100,1099,490011001001099,Block 1099,G5040,R,None,None,S,127548,0,+38.4966739,-112.6100115,"POLYGON ((-112.61207 38.49819, -112.61194 38.4..."
4,49,001,100100,1114,490011001001114,Block 1114,G5040,R,None,None,S,938564,0,+38.4555348,-112.6088292,"POLYGON ((-112.60444 38.44611, -112.60476 38.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115401,49,057,210100,3071,490572101003071,Block 3071,G5040,R,None,None,S,37660,0,+41.2598165,-111.7689839,"POLYGON ((-111.77018 41.26064, -111.77018 41.2..."
115402,49,057,210100,3090,490572101003090,Block 3090,G5040,R,None,None,S,36839,0,+41.2597387,-111.7735264,"POLYGON ((-111.77234 41.25890, -111.77280 41.2..."
115403,49,057,210100,2105,490572101002105,Block 2105,G5040,R,None,None,S,1448,0,+41.3070025,-111.8302460,"POLYGON ((-111.82974 41.30706, -111.82984 41.3..."
115404,49,057,210100,2026,490572101002026,Block 2026,G5040,R,None,None,S,105321,0,+41.3714503,-111.9042972,"POLYGON ((-111.90765 41.37064, -111.90797 41.3..."


# Join by spatial

In [13]:
import geopandas as gpd
import pandas as pd

# Load the census data from an Excel file
census_df = pd.read_excel('/Users/yingxizhao/Library/CloudStorage/GoogleDrive-yiz158@ucsd.edu/Shared drives/Econ Research Lab Fall 2023/Data/census/census data/census blocks/utah_block_2010.xlsx')
census_gdf = gpd.GeoDataFrame(census_df)

# Load the shapefile for wards
wards = gpd.read_file('/Users/yingxizhao/Library/CloudStorage/GoogleDrive-yiz158@ucsd.edu/Shared drives/Econ Research Lab Fall 2023/Data/ward boundaries/slc_area_2013_cleaned.shp')

# Use a spatial join to associate wards with the census blocks they overlap
overlap_gdf = gpd.sjoin(wards, census_gdf, how="inner", op='intersects')

# Create a GeoDataFrame to store the results
results = gpd.GeoDataFrame()

# Iterate through each ward polygon
for ward_index, ward_row in wards.iterrows():
    ward_geometry = ward_row['geometry']

    # Filter overlap_gdf for the current ward
    matching_rows = overlap_gdf[overlap_gdf.index_left == ward_index]

    # Calculate overlap fractions for each match
    overlap_fractions = [row['geometry_right'].intersection(ward_geometry).area / ward_geometry.area for index, row in matching_rows.iterrows()]

    # Add the sum of overlap fractions for this ward to the results GeoDataFrame
    results.loc[ward_index, 'overlap_fractions'] = sum(overlap_fractions)

# Optionally, save the results as a new shapefile
results.to_file('ward_census_overlap.shp')


/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


AttributeError: The CRS attribute of a GeoDataFrame without an active geometry column is not defined. Use GeoDataFrame.set_geometry to set the active geometry column.